In [1]:
from training.perceiver import*
from training.utils import*
from training.losses import*
from training.VIT import*
from training.ResNet import*
from collections import defaultdict
from training import*

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

import torch
import numpy as np
from torch import nn, einsum
import torch.nn.functional as F
import einops as einops
from einops import rearrange, repeat
from einops.layers.torch import Reduce

import matplotlib.pyplot as plt

from configilm import util
util.MESSAGE_LEVEL = util.MessageLevel.INFO  # use INFO to see all messages


from configilm.extra.DataSets import BENv2_DataSet
from configilm.extra.DataModules import BENv2_DataModule

torch.manual_seed(0)

# Perceiver

In [1]:
!pip install einops matplotlib pandas h5py pytorch-lightning "configilm[full]" scikit-learn numpy matplotlib
!pip uninstall wandb -y
!pip install wandb==0.15.12


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 352.9 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 354.7 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 285.8 kB/s eta 0:00:00a 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 309.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 356.0 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 376.2 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 307.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 368.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 282.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 413.4 kB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 406.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━

In [1]:
from training.perceiver import*
from training.utils import*
from training.losses import*
from training.VIT import*
from training.ResNet import*
from collections import defaultdict
from training import*

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

import torch
import numpy as np
from torch import nn, einsum
import torch.nn.functional as F
import einops as einops
from einops import rearrange, repeat
from einops.layers.torch import Reduce

import matplotlib.pyplot as plt

from configilm import util
util.MESSAGE_LEVEL = util.MessageLevel.INFO  # use INFO to see all messages


from configilm.extra.DataSets import BENv2_DataSet
from configilm.extra.DataModules import BENv2_DataModule

torch.manual_seed(0)


from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
torch.set_default_dtype(torch.float32)
#torch.set_float32_matmul_precision('medium' | 'high')
torch.set_float32_matmul_precision('medium' )

config_dico = read_yaml("./training/configs/config_test-ViT_XS.yaml")

bands_yaml="./data/Tiny_BigEarthNet/bands.yaml"
configs_dataset="./data/Tiny_BigEarthNet/configs_dataset.yaml"

test_conf= transformations_config(bands_yaml,configs_dataset,path_imgs_config="./data/Tiny_BigEarthNet/")

data_module=Tiny_BigEarthNetDataModule( "./data/Tiny_BigEarthNet/tiny", batch_size=config_dico['dataset']['batchsize'], num_workers=4,trans_config=test_conf,perceiver=False)

data_module.setup()
# Prepare dataloaders
train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()
test_loader = data_module.test_dataloader()

# Read configuration



tmp_name=tmp_name=get_xp_name(config_dico['encoder'])+" modalities"

#if config_dico['encoder']=="ViT":
#    tmp_name=get_xp_name(config_dico['encoder'],config_dico['encoder']["config"])
    



wand=True
wandb_logger=None
logger_xp=None#wandb_logger
if wand:
    wandb.init(
            name=tmp_name,
            project="first_xp",
            config=config_dico
        )

    wandb_logger=WandbLogger(project="first_xp")

    

dataloaders = (train_loader, val_loader, test_loader)
datasets_trainer=(data_module.train_dataset,data_module.val_dataset,data_module.test_dataset)
# Create model
model = Model(config_dico,datasets_trainer, dataloaders,wand=wand)


# Configure the trainer
trainer = Trainer(max_epochs=config_dico["trainer"]["epochs"], logger= wandb_logger,log_every_n_steps=1)



# Fit the model
trainer.fit(model, train_loader, val_loader)

# Test model with best validation loss
model.load_model(name="best_train_loss")
model.mode="best_train_loss"
trainer.test(model, test_loader)

#Test model with best validation accuracy
model.load_model(name="best_train_ap")
model.mode="best_train_ap"
trainer.test(model, test_loader)

# Test model with best validation loss
model.load_model(name="best_val_loss")
model.mode="best_val_ap"
trainer.test(model, test_loader)

#Test model with best validation accuracy
model.load_model(name="best_val_ap")
model.mode="best_val_ap"
trainer.test(model, test_loader)





Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hugordet (hugordet-inria). Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.11/site-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type              | Params | Mode 
------------------------------------------------------
0 | encoder | SimpleViT         | 4.9 M  | train
1 | loss    | BCEWithLogitsLoss | 0      | train
------------------------------------------------------
4.9 M     Trainable params
0         Non-trainable params
4.9 M     Total params
19.646    Total estimated model params size (MB)
109       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                    Test metric                    ┃                   DataLoader 0                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│             test ,modality: test mAP              │                25.878774642944336                 │
│       test ,modality: test overall accuracy       │                 86.79501342773438                 │
│             test ,modality: train mAP             │                 37.07234573364258                 │
│      test ,modality: train overall accuracy       │                 88.29085540771484                 │
│          test ,modality: validation mAP           │                27.795169830322266                 │
│    test ,modality: validation overall accuracy    │                 87.40442657470703                 │
│                     test_acc                      │                 86.79498291015625                 │
│           test_mean_precision_per_class           │                 35.81576156616211                 │
│             train ,modality: test mAP             │                 36.27234649658203                 │
│      train ,modality: test overall accuracy       │                 91.45983123779297                 │
│            train ,modality: train mAP             │                60.260337829589844                 │
│      train ,modality: train overall accuracy      │                 93.29086303710938                 │
│          train ,modality: validation mAP          │                31.385345458984375                 │
│   train ,modality: validation overall accuracy    │                 91.44044494628906                 │
│          train_mean_precision_per_class           │                 52.10134506225586                 │
│          validation ,modality: test mAP           │                26.172794342041016                 │
│    validation ,modality: test overall accuracy    │                 88.9473648071289                  │
│          validation ,modality: train mAP          │                 40.0648193359375                  │
│   validation ,modality: train overall accuracy    │                 90.45152282714844                 │
│       validation ,modality: validation mAP        │                26.610273361206055                 │
│ validation ,modality: validation overall accuracy │                 89.40719604492188                 │
│        validation_mean_precision_per_class        │                 36.99302673339844                 │
└───────────────────────────────────────────────────┴───────────────────────────────────────────────────┘

[{'test_acc': 86.79498291015625,
  'train ,modality: train overall accuracy': 93.29086303710938,
  'train ,modality: train mAP': 60.260337829589844,
  'train_mean_precision_per_class': 52.10134506225586,
  'train ,modality: validation overall accuracy': 91.44044494628906,
  'train ,modality: validation mAP': 31.385345458984375,
  'train ,modality: test overall accuracy': 91.45983123779297,
  'train ,modality: test mAP': 36.27234649658203,
  'validation ,modality: train overall accuracy': 90.45152282714844,
  'validation ,modality: train mAP': 40.0648193359375,
  'validation_mean_precision_per_class': 36.99302673339844,
  'validation ,modality: validation overall accuracy': 89.40719604492188,
  'validation ,modality: validation mAP': 26.610273361206055,
  'validation ,modality: test overall accuracy': 88.9473648071289,
  'validation ,modality: test mAP': 26.172794342041016,
  'test ,modality: train overall accuracy': 88.29085540771484,
  'test ,modality: train mAP': 37.07234573364258,
 

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7ff687fc1510>> (for post_run_cell), with arguments args (<ExecutionResult object at 7ff828f19c10, execution_count=1 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7ff828f0af10, raw_cell="from training.perceiver import*
from training.util.." store_history=True silent=False shell_futures=True cell_id=c7fb9265> result=[{'test_acc': 86.79498291015625, 'train ,modality: train overall accuracy': 93.29086303710938, 'train ,modality: train mAP': 60.260337829589844, 'train_mean_precision_per_class': 52.10134506225586, 'train ,modality: validation overall accuracy': 91.44044494628906, 'train ,modality: validation mAP': 31.385345458984375, 'train ,modality: test overall accuracy': 91.45983123779297, 'train ,modality: test mAP': 36.27234649658203, 'validation ,modality: train overall accuracy': 90.45152282714844, 'validation ,modality: train mAP': 40.0648193359375, 'va

TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given